<a href="https://colab.research.google.com/github/mubashramansoor130/Copy-of-loan_approval_prediction.ipynb/blob/main/Copy_of_loan_approval_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving loan_data_set.csv to loan_data_set.csv


In [ ]:
import pandas as pd

df = pd.read_csv('loan_data_set.csv')
display(df.head())

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Handling missing values below

In [ ]:
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])

Missing values per column:
Gender              13
Married              3
Dependents          15
Self_Employed       32
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
dtype: int64


In [ ]:
for col in df.columns:
    if df[col].isnull().any():
        if df[col].dtype in ['int64', 'float64']:
            # Impute numerical columns with the mean
            df[col].fillna(df[col].mean(), inplace=True)
        else:
            # Impute categorical columns with the mode
            df[col].fillna(df[col].mode()[0], inplace=True)

# Verify that there are no remaining missing values
print("\nMissing values after imputation:")
print(df.isnull().sum()[df.isnull().sum() > 0])


Missing values after imputation:
Series([], dtype: int64)


/tmp/ipython-input-6-1762959339.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-6-1762959339.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try

Encoding of categorical data done below

In [ ]:
for col in df.columns:
    if df[col].dtype == 'object':
        if df[col].nunique() == 2:
            # Binary encoding
            unique_values = df[col].unique()
            df[col] = df[col].map({unique_values[0]: 0, unique_values[1]: 1})
        else:
            # One-hot encoding
            df = pd.get_dummies(df, columns=[col], drop_first=True)

display(df.head())

,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,...,Loan_ID_LP002978,Loan_ID_LP002979,Loan_ID_LP002983,Loan_ID_LP002984,Loan_ID_LP002990,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Semiurban,Property_Area_Urban
0,0,0,0,0,5849,0.0,146.412162,360.0,1.0,0,...,False,False,False,False,False,False,False,False,False,True
1,0,1,0,0,4583,1508.0,128.000000,360.0,1.0,1,...,False,False,False,False,False,True,False,False,False,False
2,0,1,0,1,3000,0.0,66.000000,360.0,1.0,0,...,False,False,False,False,False,False,False,False,False,True
3,0,1,1,0,2583,2358.0,120.000000,360.0,1.0,0,...,False,False,False,False,False,False,False,False,False,True
4,0,0,0,0,6000,0.0,141.000000,360.0,1.0,0,...,False,False,False,False,False,False,False,False,False,True


Splitting of data into train and test data

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (491, 627)
Shape of X_test: (123, 627)
Shape of y_train: (491,)
Shape of y_test: (123,)


Training of Classification model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Evaluation of model performance

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.99      0.86        80
           1       0.95      0.42      0.58        43

    accuracy                           0.79       123
   macro avg       0.85      0.70      0.72       123
weighted avg       0.83      0.79      0.76       123

